In [ ]:
%pip install openai

In [ ]:
# %%writefile -a ./.env

# AZURE_OPENAI_API_KEY=
# AZURE_OPENAI_ENDPOINT=

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

from openai import AzureOpenAI
import base64

# Initialize Azure OpenAI client
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-05-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# Function to encode image to base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = "OpenAI_demo_files/the_last_supper.jpeg"
base64_image = encode_image(image_path)

# Create the message with both text and image
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "What can you see in this image? If there are any people, what are they doing?"
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{base64_image}"
                }
            }
        ]
    }
]

# Send request to GPT-4V
response = client.chat.completions.create(
    model="gpt-4o", 
    messages=messages,
    max_tokens=300
)

# Print the response
print(response.choices[0].message.content)

The image depicts a famous historical or religious scene known as "The Last Supper." The people in the painting are sitting around a long table, engaged in conversation, with various expressions and gestures. The table is set with plates, cups, and bread. The setting appears to be an indoor room with windows in the background revealing a distant landscape. The painting captures a moment of activity and interaction among the individuals.


In [3]:
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

import os
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-05-01-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

assistant = client.beta.assistants.create(
  name="Financial Analyst Assistant",
  instructions="You are an expert financial analyst. Use your knowledge base to answer questions about audited financial statements.",
  model="gpt-4o",
  tools=[{"type": "file_search"}],
)

In [4]:

# Create a vector store called "Financial Statements"
vector_store = client.beta.vector_stores.create(name="Financial Statements")
 
# Ready the files for upload to OpenAI
file_paths = ["OpenAI_demo_files/financial_results.pdf"]
file_streams = [open(path, "rb") for path in file_paths]
 
# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
 
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [5]:
thread = client.beta.threads.create()

# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What are the major source of revenue for the company?"
)

# Run the thread and poll for the result
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please answer based on files provided",
)

if run.status == "completed":
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    for message in messages.data[::-1]:
        print(f"{message.role}: {'/n'.join([x.text.value for x in message.content])}")

user: What are the major source of revenue for the company?
assistant: The major sources of revenue for the company are:

1. **Digital Media Segment**: This segment includes major products such as Creative Cloud and Document Cloud. For the second quarter of fiscal year 2024, the Digital Media segment reported a revenue of $3.91 billion, representing 11 percent year-over-year growth. Specifically, Creative Cloud revenue was $3.13 billion, and Document Cloud revenue was $782 million【4:0†source】【4:3†source】.

2. **Digital Experience Segment**: This segment includes subscription services related to the company's Experience Cloud. For the same quarter, the Digital Experience segment reported a revenue of $1.33 billion, representing 9 percent year-over-year growth, with subscription revenue accounting for $1.20 billion【4:0†source】【4:3†source】.

Overall, the company achieved a total revenue of $5.31 billion in the second quarter of fiscal year 2024, driven by strong growth across these segmen

In [6]:
message.content[0].text.annotations

[FileCitationAnnotation(end_index=426, file_citation=FileCitation(file_id='assistant-u7dsPovnpdOMyQJuAcWlwvuE'), start_index=414, text='【4:0†source】', type='file_citation'),
 FileCitationAnnotation(end_index=426, file_citation=FileCitation(file_id='assistant-u7dsPovnpdOMyQJuAcWlwvuE'), start_index=414, text='【4:0†source】', type='file_citation'),
 FileCitationAnnotation(end_index=438, file_citation=FileCitation(file_id='assistant-u7dsPovnpdOMyQJuAcWlwvuE'), start_index=426, text='【4:3†source】', type='file_citation'),
 FileCitationAnnotation(end_index=438, file_citation=FileCitation(file_id='assistant-u7dsPovnpdOMyQJuAcWlwvuE'), start_index=426, text='【4:3†source】', type='file_citation'),
 FileCitationAnnotation(end_index=765, file_citation=FileCitation(file_id='assistant-u7dsPovnpdOMyQJuAcWlwvuE'), start_index=753, text='【4:0†source】', type='file_citation'),
 FileCitationAnnotation(end_index=765, file_citation=FileCitation(file_id='assistant-u7dsPovnpdOMyQJuAcWlwvuE'), start_index=753, 

In [7]:

# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Can you elaborate on source 2?"
)

# Run the thread and poll for the result
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please answer based on files provided",
)

if run.status == "completed":
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    for message in messages.data[::-1]:
        print(f"{message.role}: {'/n'.join([x.text.value for x in message.content])}")

user: What are the major source of revenue for the company?
assistant: The major sources of revenue for the company are:

1. **Digital Media Segment**: This segment includes major products such as Creative Cloud and Document Cloud. For the second quarter of fiscal year 2024, the Digital Media segment reported a revenue of $3.91 billion, representing 11 percent year-over-year growth. Specifically, Creative Cloud revenue was $3.13 billion, and Document Cloud revenue was $782 million【4:0†source】【4:3†source】.

2. **Digital Experience Segment**: This segment includes subscription services related to the company's Experience Cloud. For the same quarter, the Digital Experience segment reported a revenue of $1.33 billion, representing 9 percent year-over-year growth, with subscription revenue accounting for $1.20 billion【4:0†source】【4:3†source】.

Overall, the company achieved a total revenue of $5.31 billion in the second quarter of fiscal year 2024, driven by strong growth across these segmen